# Overview

This is for the second round of testing.  **Updated 26 July 2022**. 

- solved: retrieving multiple pages from Airtable
- Next: using email to try to find individuals.  What do we do if an individual email maps to multiple companies?

In [2]:
import pandas as pd
import numpy as np
import os
import json 
import requests
from airtable import airtable #You need to do a pip install airtable.  Page at https://pypi.org/project/airtable/

In [3]:
#Read the secrets file
def read_secrets(fn) -> dict:
    filename = os.path.join(fn)
    try:
        with open(filename, mode='r') as f:
            return json.loads(f.read())
    except FileNotFoundError:
        return {}

In [4]:
#note the API base id is specific to the Airtable "base".  So, adjust the secrets and the ID to fit the specific Airtable base.
#the api key is an account-wide value so shouldn't change.
secret = read_secrets('secrets2.json')

In [5]:
#Start by downloading data from Airtable
#https://towardsdatascience.com/downloading-airtable-data-into-python-89e5c7107a24
#reading participant database - Jayson copy.  Will need to change table id in secrets.json to use the real one.

#Get a table in the Airtable base
#see https://github.com/josephbestjames/airtable.py

at = airtable.Airtable(secret['AIRTABLE_BASE_ID'], secret['AIRTABLE_API_KEY'])

In [40]:
#Note, only returns 1 page = 100 records per request.  So, we'll need to do pagination.

#Basically I want to write a recursion to make the next call until the current dict size is x<100, then exit.
#But, before that, I'll get two pages and merge?  Or, should I work with them separately?  Could do either, maybe I don't need to merge.

def get_all_records(pages,offset=None,count=0):
    
    count = count + 1
    at_dict = at.get(secret['AIRTABLE_TABLE_ID'],offset=offset) 
    
    if 'offset' in at_dict:
        offset = at_dict['offset']
        pages.append(at_dict) 
        if count % 10 ==0:
            print(count,offset) #this is for debugging.  Remove in production version.
            
    else: #if there is no offset, that means that we hit the last page
        pages.append(at_dict) 
        print(count,offset)
        return(pages)
        exit
        
    return(get_all_records(offset = offset,pages = pages,count=count))

In [41]:
#For now, need to pass these blank lists.  Might be a better way, like tuples or something.
pages = []
all_pages = get_all_records(pages=pages)

10 itrGEJzmkVTn4y4ZW/rec5CIPLxq6bYu481
20 itrGEJzmkVTn4y4ZW/recAF6s9I2j3mzhvS
30 itrGEJzmkVTn4y4ZW/recEwzqnOjgu2o0KG
40 itrGEJzmkVTn4y4ZW/recJjydwQWSRIbgiI
50 itrGEJzmkVTn4y4ZW/recOcfKDipC4bsfHy
60 itrGEJzmkVTn4y4ZW/recTL79eQvCaxxuyl
70 itrGEJzmkVTn4y4ZW/recYDsbriissrNlUZ
80 itrGEJzmkVTn4y4ZW/recdNvox9c8RmC5oR
90 itrGEJzmkVTn4y4ZW/reciOH1mNu64vljT6
100 itrGEJzmkVTn4y4ZW/recnTkESbLehVRbdx
110 itrGEJzmkVTn4y4ZW/recsBWxnI0rkacrM7
120 itrGEJzmkVTn4y4ZW/recxECmofMwounhKc
126 itrGEJzmkVTn4y4ZW/reczl4Q8FE1JF0r91


In [50]:
L = len(all_pages)
print(f'{L} pages returned')
p = 0
for i in range(L):
    p = p + len(all_pages[i]['records'])
    
print(f'{p} records total')

all_pages[125]['records']

126 pages returned
12546 records total


[OrderedDict([('id', 'reczl5EkayQMgvJOM'),
              ('createdTime', '2022-07-25T20:31:22.000Z'),
              ('fields',
               OrderedDict([('Company Revenue', []),
                            ('Record ID', 'reczQsM2JHmR14NCg'),
                            ('Voice Channels', ['']),
                            ('Segment Personas Features', []),
                            ('Name', 'Kim Young'),
                            ('Job Title', 'Owner'),
                            ('SFDC Billing Country', 'United States'),
                            ('Industry', []),
                            ('Responsibility level: Driving retention/repeat purchase (from Email)',
                             []),
                            ('Programmable Voice Channels', ['']),
                            ('Years Coding', 'Less than 1 year'),
                            ('Preferred Programming language (from Filter survey responses)',
                             []),
                       

In [ ]:
# for i in range(rec):
#     f = at_dict['records'][i]['fields']
#     id = at_dict['records'][i]['id']
#     n = at_dict['records'][i]['fields']['Email']
#     if n == 'ayushman.sinhaa@gmail.com':
#         print(i,id)

In [ ]:
#A field to update
#update_data = {"Email": "updated@viaAPI.com"}
#update_data

In [ ]:
#Update the email address for a given record based on the record id (generated by AirTable)
#Here, the id was obtained by matching the record id for name 'Nthabi Choma'
#at.update(secret['AIRTABLE_TABLE_ID'], id, update_data)